In [1]:
%cd /home/jovyan/work

from bokeh.io import output_notebook

output_notebook()


/home/jovyan/work


Loading BokehJS ...

In [2]:
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.util.hex import axial_to_cartesian
from bokeh.models import HoverTool

from chordifier.KeyboardRenderer import KeyboardRenderer
from chordifier.experiments.KeyboardFactory import make

def present(keyboard):
    renderer = KeyboardRenderer(keyboard)

    f = make_figure()
    draw_hexagons(f, renderer.qs, renderer.rs, renderer.colors, renderer.positions)
    return f

def make_figure():
   
    f = figure(tools="pan,wheel_zoom,reset", height=600,
               sizing_mode='stretch_width', match_aspect=True)
    f.grid.visible = False
    quad = f.quad(top=[15], bottom=[-16], left=[-25],
       right=[9], color="#000000", alpha=0)
    
    hover = HoverTool()
    hover.tooltips = [
        ("x", "$x"),
        ("y", "$y"),
    ]
    hover.renderers = [quad]
    hover.point_policy = "follow_mouse"
    hover.attachment = "right"
    f.add_tools(hover)

    return f

def draw_hexagons(f, q, r, colors, labels):
    f.hex_tile(q, r, fill_color=colors, alpha=0.5, line_color="white",
               orientation="flattop")

    x, y = axial_to_cartesian(q, r, 1, "flattop")

    f.text(x, y, text=labels,
           text_baseline="middle", text_align="center")


In [3]:
from chordifier.algorithm.Pipeline import Pipeline
from chordifier.experiments.KeyboardFactory import make
from chordifier.utils import vector

PARAMETERS = {
    # general
    'keyboard': make([3, 3, 3, 3, 2] + [0] * 5),
    'characters': 20,

    # sequencer
    'filename': "dataset/ngram.json",
    'length': 3,
    'samples': 500,

    # dynamics
    'x_y_ratio': 1.5,
    'stiffness': vector(1.8, 1.3, 1.2, 1.1, 2, 2, 1.1, 1.2, 1.3, 1.8),

    # pruner
    'priority': vector(1800000, 1300000, 1200000, 1100000, 1450000,
                       1451600, 1100800, 1200400, 1300200, 1800100),
    'finger_priorities': 1,
    'average_offsets': 0,
    'deviation_of_offsets': 0,

    # evaluator
    'distances_travelled': 1,
    'chord_difficulties': 1,

    # optimizer
    'best': True,
    'generations': 10,
    'population_size': 100,
    'mate_probability': 0.5,
    'mutate_probability': 0.20,
    'mutate_independent_probability': 0.10,
    'select_tournament_size': 10,
}

pipeline = Pipeline(PARAMETERS)

pipeline.prepare()
result = pipeline.optimize()


gen	nevals	average    	standard   	minimum    	maximum    
0  	100   	7.30967e+08	9.94255e+07	5.58809e+08	1.07312e+09
1  	63    	6.38639e+08	7.57439e+07	5.35136e+08	9.60504e+08
2  	52    	6.13169e+08	7.63806e+07	5.13218e+08	8.76041e+08
3  	62    	5.88033e+08	7.19257e+07	5.10793e+08	8.45458e+08
4  	52    	5.79552e+08	1.05407e+08	5.10793e+08	9.72835e+08
5  	67    	5.43831e+08	8.32621e+07	4.98341e+08	1.00764e+09
6  	65    	5.6534e+08 	1.01772e+08	4.98341e+08	1.00764e+09
7  	60    	5.32679e+08	5.13331e+07	4.90222e+08	7.50182e+08
8  	55    	5.38052e+08	8.07189e+07	4.8036e+08 	8.35531e+08
9  	66    	5.65808e+08	1.09579e+08	4.60552e+08	9.15521e+08
10 	62    	5.51435e+08	8.91961e+07	4.60552e+08	7.99475e+08


In [4]:
print("=== Keyboard ===")
f = present(pipeline.keyboard)
show(f)


=== Keyboard ===


In [5]:
print("=== Preprocessor ===")
print("--- zones ===")
for zone in pipeline.preprocessor.zones:
    print(zone)
print("--- chords ===")
print(pipeline.preprocessor.chords[222:230:4])
print("--- positions ===")
print(pipeline.preprocessor.positions[222:230:4])
print("--- origins ===")
print(pipeline.preprocessor.origins)


=== Preprocessor ===
--- zones ===
[[-8  4]
 [-8  3]
 [-9  4]
 [-9  5]
 [-8  5]]
[[-7  3]
 [-7  2]
 [-7  4]]
[[-6  2]
 [-6  1]
 [-6  3]]
[[-5  2]
 [-5  1]
 [-4  1]
 [-4  2]
 [-5  3]]
[[-3  3]
 [-3  2]
 [-2  2]
 [-2  3]
 [-3  4]
 [-4  4]
 [-4  3]]
[]
[]
[]
[]
[]
--- chords ===
[[0 1 0 3 7 0 0 0 0 0]
 [0 1 0 4 3 0 0 0 0 0]]
--- positions ===
[[[         nan          nan]
  [-10.5          0.8660254 ]
  [         nan          nan]
  [ -6.           1.73205081]
  [ -6.          -1.73205081]
  [         nan          nan]
  [         nan          nan]
  [         nan          nan]
  [         nan          nan]
  [         nan          nan]]

 [[         nan          nan]
  [-10.5          0.8660254 ]
  [         nan          nan]
  [ -6.          -0.        ]
  [ -3.          -1.73205081]
  [         nan          nan]
  [         nan          nan]
  [         nan          nan]
  [         nan          nan]
  [         nan          nan]]]
--- origins ===
[[-12.          -0.        ]
 [-10.5  